# Day 7

## GENERIC SETUP

In [1]:
# General imports
import pytest
import ipytest
import time
import functools

# Setup ipytest
ipytest.autoconfig()

# Setup nb_black
%load_ext nb_black

# Decorator to time solutions
def timer(func):
    """
    Wrapper function.
    Print the runtime of the decorated function.
    """

    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()  # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()  # 2
        run_time = end_time - start_time  # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value

    return wrapper_timer

<IPython.core.display.Javascript object>

## SOLUTION SETUP

In [2]:
# Solution-specific imports
import re

# What day do we solve? Used to identify the input datafile, integer value
DAY = 7

<IPython.core.display.Javascript object>

#### I/O functions

In [3]:
def get_input():
    with open(f"../data/{DAY}.txt", "r") as f:
        return split_input(f.read())


def split_input(input_raw):
    """Strip trailing newline, then split on newline"""
    return (line.strip() for line in input_raw.rstrip().split("\n"))

<IPython.core.display.Javascript object>

#### Pytest input data

In [4]:
# Sample input
@pytest.fixture
def dummy_input():
    return """\
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
"""

<IPython.core.display.Javascript object>

## Solution A

In [5]:
def parse_bag(line):
    """
    Read a single line, extracting the colour and allowed contents.
    Returns a tuple of (colour, allowed content dict)
    """
    # 1. Retrieve the colour
    bag_colour = re.match("^\w+ \w+", line).group(0)

    # 2. Split the bag contents into strings
    bag_content_raw = re.findall("\d+ \w+ \w+", line)

    # 3. Parse each entry, add to bag_content list
    bag_content = (
        []
    )  # Alternative: dictionary from content colour to number of that colour
    for item in bag_content_raw:
        # Add the colour to the list of bag_content colours
        # Ignore the number of bags for now
        bag_content.append(re.match("\d+ (\w+ \w+)", item).group(1))

    return bag_colour, bag_content


def search_gold(bags, colour):
    # If found
    if "shiny gold" in bags[colour]:
        return True
    # If empty
    elif len(bags[colour]) < 1:
        return False
    # Search deeper, return True if any contained bag returned True
    else:
        return any([search_gold(bags, x) for x in bags[colour]])


@timer
def solve_A(lines):
    """
    1. Parse each line to get colour and [contents],
    2. turn into a dict
    3. For key in dict (except shiny gold itself), search gold recursively
    4. Return number of bags containing gold
    """
    # 1. Generate a list of (colour, content) tuples for each line
    parsed_lines = [parse_bag(line) for line in lines]
    # 2. Create a dictionary from the list
    bags_dict = dict((k, v) for k, v in parsed_lines)

    # 3. Now, for each key in bags_dict, check for shiny gold is possible
    gold_inside = [
        search_gold(bags_dict, colour)
        for colour in bags_dict.keys()
        if colour != "shiny gold"
    ]
    # 4. Return the number of bag colours than can contain gold
    return sum(gold_inside)

<IPython.core.display.Javascript object>

#### Tests

In [6]:
%%run_pytest[clean] -qq

def test_A(dummy_input):
    assert solve_A(split_input(dummy_input)) == 4

<IPython.core.display.Javascript object>

.                                                                                                                                                                                                                                      [100%]


<IPython.core.display.Javascript object>

#### OUTPUT

In [7]:
solve_A(get_input())

Finished 'solve_A' in 0.1230 secs


370

<IPython.core.display.Javascript object>

## Solution B

In [8]:
def parse_bag_B(line):
    """
    Read a single line, extracting the colour and allowed contents.

    Returns a tuple of (colour, allowed content dict)
    """
    # 1. Retrieve the colour
    bag_colour = re.match("^\w+ \w+", line).group(0)

    # 2. Split the bag contents into strings
    bag_content_raw = re.findall("\d+ \w+ \w+", line)

    # 3. Parse each entry, add to bag_content list
    bag_content = {}
    # For each item in contents_raw
    for item in bag_content_raw:
        # Get the colour and number of bags
        bag_content[re.match("\d+ (\w+ \w+)", item).group(1)] = int(item.split(" ")[0])

    return bag_colour, bag_content


def count_bags(bags, colour):
    """Count the number of bags inside bag of colour [colour]"""
    if bags[colour] == {}:
        return 0
    # How many bags are contained directly
    bags_inside = sum(bags[colour].values())
    # How many bags are contained recursively
    bags_recursively = [
        num * count_bags(bags, bag) for bag, num in bags[colour].items()
    ]
    # Return the sum of recursively found bags plus bags directly inside
    return sum(bags_recursively) + bags_inside


@timer
def solve_B(lines):
    """
    1. Parse each line to get colour and {content: number} dict,
    2. turn into a dict of bags and content dicts
    3. For shiny gold, count contained bags recursively
    4. Return number of bags inside of shiny gold
    """
    # 1. Generate a list of (colour, content) tuples for each line
    parsed_lines = [parse_bag_B(line) for line in lines]
    # 2. Create a dictionary from the list
    bags_dict = dict((k, v) for k, v in parsed_lines)

    # 3. Count contained bags recursively in 'shiny gold'
    bags_in_shiny_gold = count_bags(bags_dict, "shiny gold")

    return bags_in_shiny_gold

<IPython.core.display.Javascript object>

#### Tests

In [9]:
%%run_pytest[clean] -qq

def test_B(dummy_input):
    assert solve_B(split_input(dummy_input)) == 32

<IPython.core.display.Javascript object>

.                                                                                                                                                                                                                                      [100%]


<IPython.core.display.Javascript object>

#### OUTPUT

In [10]:
solve_B(get_input())

Finished 'solve_B' in 0.0051 secs


29547

<IPython.core.display.Javascript object>